**IBM Data Science Professional Certificate: Capstone Project Report**


**Author: Zicao Fu**  
**Email: zicao.fu@gmail.com**  
**May 2020**

# Where is your familiar working and living place?

**A recommendation system for finding neighborhoods with similar venues in a new city**

This project aims to build a **recommendation system** which helps people to find neighborhoods in a new city with venues similar to those of their old living places.

Here I use myself as an example and try to look for neighborhoods in **New York** and **Toronto** that are most similar to my current working and living place, **Sha Tin district of Hong Kong**. Venues around a neighborhood are obtained from **Foursquare** and are then used to construct a feature set which characterizes that particular neighborhood. Afterwards, different neighborhoods in the new cities are compared to my current place based on their respective feature sets, and those that share most similarities with my current place are recommended and get marked on an interactive **Folium map**.

From the total number of recommended neighborhoods in a particular city, one can also see which city on the whole most resembles my current place, at least as far as living is concerned. In the case studied here, we will see whether it is New York or Toronto that is most similar to the Sha Tin district of Hong Kong.

## 1. Introduction

### 1.1 Business Background

We currently live in an age of globalization. People travel around the world more frequently than in the past. More and more people face the problem of finding a neighborhood in a foreign city to live and work. Moving to a new neighborhood which contains many venues similar to his/her old working and living place can save the time and cost needed to adapt to the new environment. Thus, the issue of finding neighborhoods with similar venues in a new city becomes more and more relevant to many people. However, it is not a easy task, especially when people need to make their decisions remotely and even before they first arrive the new city.

This is the place where data science can help. Utilizing the vast amount of location and venue information available online, if we can build a feature set for every neighborhood in the cities concerned, we can then use these feature sets to compare different neighborhoods with the customer’s current working and living place (or any other places he/she picks) and come up with the top neighborhoods in those cities that most resemble this place. The result can aid customers to make a well-informed choice in deciding where to work and live.

As an analogy, the recommendation system which will be built here is similar in essence to that used by Amazon which suggests the next commodity for you to buy.

### 1.2 Business Problem

In this article, I use myself as an example for this project. Currently, I work and live in **Sha Tin district of Hong Kong**, which is the most populous district in Hong Kong and is away from the city center of Hong Kong. I will use the subdistricts in this district as a benchmark in my search for similar neighborhoods in two cities on the other side of the globe.

In Hong Kong, the words "borough" and "neighborhood" are not used to describe regions in the city. Instead, the words "district" and "subdistrict" are used. In this article, "district" and "borough" are used interchangeably. The two words "subdistrict" and "neighborhood" are used interchangeably as well.

Since my career goal is to apply data science to financial service industry, the next city which I might move into probably will be a financial center. I pick a major financial center in the US and a major financial center in Canada to do the battle (comparison) of neighborhoods. The two cities chosen in this study are **New York** and **Toronto**. The question which this project answers is stated as: "**Which 20 neighborhoods in New York and Toronto do have sets of venues most similar to those in Sha Tin district of Hong Kong?**"

**The main objective is to recommend a total number of 20 neighborhoods in these two cities.** From the recommendation, we can also learn which city has the largest number of recommended neighborhoods, thus most similar to my current working and living place, Sha Tin district of Hong Kong. Choosing a right neighborhood is a daunting task. Therefore, the analysis done here would be interesting to **anyone who plans to move into a new city**.

In [1]:
# In this cell, I import packages.

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 102kB 11.4MB/s ta 0:00:01
Libraries imported.


## 2. Data

Based on the business problem described above, there are two data sets needed. The first data set contains data of neighborhoods with latitudes and longitudes in concerned cities. The second data set contains data of the information of venues in concerned neighborhoods.

### 2.1 Neighborhood Location Data

For the first data set, we need data in the form of tables with five columns: city, borough, neighborhood, latitude, longitude. We need three such tables, respectively for Sha Tin district of Hong Kong, New York, and Toronto. In addition, in order to make data processing easy, we will create a master data frame \texttt{TwoCities} for the two tables for New York and Toronto combined and a grand master data frame \texttt{ThreeCities} for all three tables combined.

#### 2.1.1 Sha Tin, Hong Kong

The list of subdistricts of Sha Tin can be found at the document "Areas and Districts" published by Hong Kong government at https://www.rvd.gov.hk/doc/tc/hkpr13/06.pdf. Since there are only six neighborhoods, the data frame can be created by inputting the values by hand. Below is the full resulting data frame, ```Sha_Tin_Data```, where all the latitude and longitude data are obtained from Google map.

In [2]:
Sha_Tin_Data = pd.DataFrame(columns=['City', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'])

Sha_Tin_Neigh_name = ['Tai Wai', 'Sha Tin', 'Fo Tan', 'Ma Liu Shui', 'Wu Kai Sha', 'Ma On Shan']
Sha_Tin_Neigh_lat = [22.375944, 22.382165, 22.398566, 22.418777, 22.427887, 22.424187]
Sha_Tin_Neigh_lon = [114.178626, 114.188160, 114.192940, 114.204337, 114.236608, 114.231480]

for i in range(len(Sha_Tin_Neigh_name)):
    Sha_Tin_Data = Sha_Tin_Data.append({'City': 'Hong Kong',
                                    'Borough': 'Sha Tin',
                                    'Neighborhood': Sha_Tin_Neigh_name[i],
                                    'Latitude': Sha_Tin_Neigh_lat[i],
                                    'Longitude': Sha_Tin_Neigh_lon[i]}, ignore_index=True)
Sha_Tin_Data

,City,Borough,Neighborhood,Latitude,Longitude
0,Hong Kong,Sha Tin,Tai Wai,22.375944,114.178626
1,Hong Kong,Sha Tin,Sha Tin,22.382165,114.188160
2,Hong Kong,Sha Tin,Fo Tan,22.398566,114.192940
3,Hong Kong,Sha Tin,Ma Liu Shui,22.418777,114.204337
4,Hong Kong,Sha Tin,Wu Kai Sha,22.427887,114.236608
5,Hong Kong,Sha Tin,Ma On Shan,22.424187,114.231480


In [3]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Sha_Tin_Data['Borough'].unique()),
        Sha_Tin_Data.shape[0]
    )
)

The dataframe has 1 boroughs and 6 neighborhoods.


In [4]:
# Get the latitude and longitude of Sha Tin, Hong Kong, using geopy library.
address_ST = 'Sha Tin, Hong Kong'

geolocator_ST = Nominatim(user_agent="Hong_Kong_explorer")
location_ST = geolocator_ST.geocode(address_ST)
latitude_ST = location_ST.latitude
longitude_ST = location_ST.longitude
print('The geograpical coordinate of Sha Tin, Hong Kong are {}, {}.'.format(latitude_ST, longitude_ST))

The geograpical coordinate of Sha Tin, Hong Kong are 22.3835565, 114.1899848.


In [5]:
# create map of Hong Kong using latitude and longitude values
map_ShaTin = folium.Map(location=[latitude_ST, longitude_ST], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(Sha_Tin_Data['Latitude'], Sha_Tin_Data['Longitude'], Sha_Tin_Data['Borough'], Sha_Tin_Data['Neighborhood']):
    label = '{}: {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ShaTin)  
    
map_ShaTin

Above is a Folium map centered at Sha Tin, Hong Kong, with blue dots denoting the six neighborhoods.

#### 2.1.2 New York

We have already created a location data frame for New York in our example project of exploring neighborhoods in New York. There, the raw data was fetched from the json file at https://cocl.us/new_york_dataset. I simply repeat the procedures in that example project. The first five rows in the location data frame for New York, ```NYLocation```, is shown below. The data frame has five boroughs and 306 neighborhoods.

In [6]:
url = 'https://cocl.us/new_york_dataset'
newyork_data = requests.get(url).json()
NY_data = newyork_data['features']
NY_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [7]:
# define the dataframe columns
column_names = ['City', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
NYLocation = pd.DataFrame(columns=column_names)
NYLocation


for data in NY_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    NYLocation = NYLocation.append({'City': 'New York',
                                          'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

NYLocation.head()

,City,Borough,Neighborhood,Latitude,Longitude
0,New York,Bronx,Wakefield,40.894705,-73.847201
1,New York,Bronx,Co-op City,40.874294,-73.829939
2,New York,Bronx,Eastchester,40.887556,-73.827806
3,New York,Bronx,Fieldston,40.895437,-73.905643
4,New York,Bronx,Riverdale,40.890834,-73.912585


In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(NYLocation['Borough'].unique()),
        NYLocation.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [9]:
# Get the latitude and longitude of New York, using geopy library.
address_NY = 'New York City, NY'

geolocator_NY = Nominatim(user_agent="newyork_explorer")
location_NY = geolocator_NY.geocode(address_NY)
latitude_NY = location_NY.latitude
longitude_NY = location_NY.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude_NY, longitude_NY))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [10]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude_NY, longitude_NY], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(NYLocation['Latitude'], NYLocation['Longitude'], NYLocation['Borough'], NYLocation['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Above is a Folium map centered at New York city, with blue dots denoting all the neighborhoods in New York city.

#### 2.1.3 Toronto

We have already created a location data frame in our exercise project of segmenting and clustering neighborhoods in Toronto. There, we obtain the data of boroughs and neighborhoods by scraping the Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and we obtain the latitude and longitude data at https://cocl.us/Geospatial_data. I repeat the procedures in that exercise project. The first five rows in the location data frame for Toronto, ```Toronto_Data```, is shown below.

In [11]:
# I scrape the Wikipedia page.
Toronto_Data_1 = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

# I ignore cells with 'not assigned' borough.
Toronto_Data_2 = Toronto_Data_1[Toronto_Data_1['Borough']!='Not assigned']

# I reset the index of the data frame.
Toronto_Data_3 = Toronto_Data_2.reset_index().drop(labels='index', axis=1)

# I use ',' symbol to replace ' /' symbol in the 'Neighborhood' column.
Toronto_Data_3['Neighborhood'] = Toronto_Data_3['Neighborhood'].str.replace(' /', ',')

# I use the provided csv file to create the geo_data data frame.
geo_data = pd.read_csv('https://cocl.us/Geospatial_data')

# In this cell, I join the two data frames together.
Toronto_Data = Toronto_Data_3.join(geo_data.set_index('Postal Code'), on='Postal Code')

# I use 'city' column to replace the 'Postal Code' column.
Toronto_Data.rename(columns={'Postal Code':'City'}, inplace=True)
CityList = []
for i in range(Toronto_Data.shape[0]):
    CityList.append('Toronto')
Toronto_Data['City'] = CityList

# I view the data.
Toronto_Data.head(15)

,City,Borough,Neighborhood,Latitude,Longitude
0,Toronto,North York,Parkwoods,43.753259,-79.329656
1,Toronto,North York,Victoria Village,43.725882,-79.315572
2,Toronto,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,Toronto,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Toronto,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,Toronto,Etobicoke,Islington Avenue,43.667856,-79.532242
6,Toronto,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,Toronto,North York,Don Mills,43.745906,-79.352188
8,Toronto,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,Toronto,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_Data['Borough'].unique()),
        Toronto_Data.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [13]:
# Get the latitude and longitude of Toronto, using geopy library.
address_TRT = 'Toronto, Canada'

geolocator_TRT = Nominatim(user_agent="Toronto_explorer")
location_TRT = geolocator_TRT.geocode(address_TRT)
latitude_TRT = location_TRT.latitude
longitude_TRT = location_TRT.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_TRT, longitude_TRT))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [14]:
# create map of New York using latitude and longitude values
map_TRT = folium.Map(location=[latitude_TRT, longitude_TRT], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_Data['Latitude'], Toronto_Data['Longitude'], Toronto_Data['Borough'], Toronto_Data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TRT)  
    
map_TRT

Above is a Folium map centered at Toronto city, with blue dots denoting all the neighborhoods in Toronto city.

Here, I would like to point out that we have obtained the data frame ```Toronto_Data``` by scraping the Wikipedia page of a list of postal codes in Canada where the first letter is M. There are cases where several different neighborhoods share the same postal code. In each of these cases, the several different neighborhoods have been treated as one single big neighborhood in our data frame ```Toronto_Data```. For example, Regent Park and Harbourfront are two different neighborhoods sharing the same postal code, thus being treated as a single big neighborhood in our data frame ```Toronto_Data```. This is not a concern for me because I do not need very fine results. Our data frame ```Toronto_Data``` has ten boroughs and 103 neighborhoods.

#### 2.1.4 Summary

Finally, we create the data frame ```TwoCities``` by merging the data of New York and Toronto together. The data frame ```TwoCities``` has 306+103=409 rows. Further, we create the data frame ```ThreeCities``` by merging ```TwoCities``` and the data of Sha Tin, Hong Kong. The data frame ```ThreeCities``` has 409+6=415 rows. Creating these two master data frames will greatly facilitates later data processing.

New York has 306 neighborhoods and Toronto has 103 neighborhoods. The number of neighborhoods in New York is roughly three times the number of neighborhoods in Toronto. Thus, if the 20 recommended neighborhoods are randomly distributed in these 409 neighborhoods, the number of recommended neighborhoods in New York should be about three times the number of recommended neighborhoods in Toronto. This should be keep in mind for later discussion.

In [15]:
TwoCities = NYLocation.append(Toronto_Data, ignore_index=True)
print('The shape of TwoCities is: ', TwoCities.shape)

The shape of TwoCities is:  (409, 5)


In [16]:
ThreeCities = TwoCities.append(Sha_Tin_Data, ignore_index=True)
print('The shape of ThreeCities is: ', ThreeCities.shape)

The shape of ThreeCities is:  (415, 5)


In [17]:
ThreeCities

,City,Borough,Neighborhood,Latitude,Longitude
0,New York,Bronx,Wakefield,40.894705,-73.847201
1,New York,Bronx,Co-op City,40.874294,-73.829939
2,New York,Bronx,Eastchester,40.887556,-73.827806
3,New York,Bronx,Fieldston,40.895437,-73.905643
4,New York,Bronx,Riverdale,40.890834,-73.912585
5,New York,Bronx,Kingsbridge,40.881687,-73.902818
6,New York,Manhattan,Marble Hill,40.876551,-73.910660
7,New York,Bronx,Woodlawn,40.898273,-73.867315
8,New York,Bronx,Norwood,40.877224,-73.879391
9,New York,Bronx,Williamsbridge,40.881039,-73.857446


### 2.2 Venue Information Data

We use Foursquare API to fetch the venue information data, same as what we did in a previous exercise project. In order to get the same number of venue categories for each neighborhood, we need to fetch the venue data for all three cities at the same time. Thus, we use our grand master ```ThreeCities``` data frame here. We store the venue information data which we obtain from Foursquare in a data frame named ```ThreeCities_venues```. The first a few rows of this data frame is shown below. It has a total of 12001 rows, which is the total number of returned venues in all three cities. There’re 461 unique venue categories.

In [18]:
# The code was removed by Watson Studio for sharing.

This cell is a hidden cell. In this cell, CLIENT_ID, CLIENT_SECRET, and VERSION are inputted.


In [19]:
def getNearbyVenues(city, borough, neighborhood, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for city, borough, neighborhood, lat, lng in zip(city, borough, neighborhood, latitudes, longitudes):
        print('CITY: ' + str(city) + ', BOROUGH: ' + str(borough) + ', NEIGHBORHOOD: ' + str(neighborhood))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            city, 
            borough, 
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City',
                  'Borough',
                  'Neighborhood',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
ThreeCities_venues = getNearbyVenues(city=ThreeCities['City'],
                                   borough=ThreeCities['Borough'],
                                   neighborhood=ThreeCities['Neighborhood'],
                                   latitudes=ThreeCities['Latitude'],
                                   longitudes=ThreeCities['Longitude']
                                  )

CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: Wakefield
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: Co-op City
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: Eastchester
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: Fieldston
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: Riverdale
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: Kingsbridge
CITY: New York, BOROUGH: Manhattan, NEIGHBORHOOD: Marble Hill
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: Woodlawn
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: Norwood
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: Williamsbridge
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: Baychester
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: Pelham Parkway
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: City Island
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: Bedford Park
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: University Heights
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: Morris Heights
CITY: New York, BOROUGH: Bronx, NEIGHBORHOOD: Fordham
CITY: New Yo

In [21]:
# Inspect our dataframe venues
print('The shape of venues is: ', ThreeCities_venues.shape)
ThreeCities_venues.head(20)

The shape of venues is:  (12001, 9)


,City,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,New York,Bronx,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,New York,Bronx,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,New York,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,New York,Bronx,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop
5,New York,Bronx,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
6,New York,Bronx,Wakefield,40.894705,-73.847201,Subway,40.890468,-73.849152,Sandwich Place
7,New York,Bronx,Wakefield,40.894705,-73.847201,Koss Quick Wash,40.891281,-73.849904,Laundromat
8,New York,Bronx,Co-op City,40.874294,-73.829939,Capri II Pizza,40.876374,-73.829940,Pizza Place
9,New York,Bronx,Co-op City,40.874294,-73.829939,Dollar Tree,40.870125,-73.828989,Discount Store


We then group rows by neighborhood. After that, we take the mean of the frequency of occurrence of each category to obtain the data frame ```venues_grouped```. First five rows and first 14 columns are shown below. The data frame ```venues_grouped``` has 406 rows, which means that 409-406=3 neighborhoods are lost. This means that Foursquare did not return any data of these three neighborhoods. We just delete these three neighborhoods from our analysis.

In [22]:
# Find out how many unique categories can be curated from all the returned venues
print('There are {} uniques venue categories.'.format(len(ThreeCities_venues['Venue Category'].unique())))

There are 459 uniques venue categories.


In [23]:
# Check how many venues were returned for each neighborhood
print("There're {} neighborhoods returned by Foursquare.".format(ThreeCities_venues.groupby(['City', 'Borough', 'Neighborhood']).count().shape[0]))
ThreeCities_venues.groupby(['City', 'Borough', 'Neighborhood']).count()

There're 406 neighborhoods returned by Foursquare.


Neighborhood Latitude  \
City      Borough          Neighborhood                                                                
Hong Kong Sha Tin          Fo Tan                                                                  6   
                           Ma Liu Shui                                                             5   
                           Ma On Shan                                                             15   
                           Sha Tin                                                                60   
                           Tai Wai                                                                25   
                           Wu Kai Sha                                                              6   
New York  Bronx            Allerton                                                               30   
                           Baychester                                                             22   
                           Bedford Park                                                           36   
                           Belmont                                                                97   
                           Bronxdale                                                              15   
                           Castle Hill                                                             8   
                           City Island                                                            26   
                           Claremont Village                                                      16   
                           Clason Point                                                            9   
                           Co-op City                                                             16   
                           Concourse                                                              28   
                           Concourse Village                                                      39   
                           Country Club                                                            8   
                           East Tremont                                                           18   
                           Eastchester                                                            20   
                           Edenwald                                                                4   
                           Edgewater Park                                                         21   
                           Fieldston                                                               3   
                           Fordham                                                                84   
                           High  Bridge                                                           24   
                           Hunts Point                                                            11   
                           Kingsbridge                                                            66   
                           Kingsbridge Heights                                                    30   
                           Longwood                                                                8   
                           Melrose                                                                26   
                           Morris Heights                                                         10   
                           Morris Park                                                            22   
                           Morrisania                                                             25   
                           Mott Haven                                                             20   
                           Mount Eden                                                             33   
                           Mount Hope                                                             12   
                       

In [24]:
# one hot encoding
venues_onehot = pd.get_dummies(ThreeCities_venues[['Venue Category']], prefix="", prefix_sep="")

# add city, borough, neighborhood columns back to dataframe
venues_onehot['City'] = ThreeCities_venues['City'] 
venues_onehot['Borough'] = ThreeCities_venues['Borough'] 
venues_onehot['Neighborhood'] = ThreeCities_venues['Neighborhood'] 

# move city, borough, neighborhood columns to the first three columns
cols = venues_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('City')))
cols.insert(1, cols.pop(cols.index('Borough')))
cols.insert(2, cols.pop(cols.index('Neighborhood')))
venues_onehot = venues_onehot.reindex(columns=cols)

print('The shape of venues_onehot is: ', venues_onehot.shape)
venues_onehot.head()

The shape of venues_onehot is:  (12001, 461)


,City,Borough,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Me

In [25]:
venues_grouped = venues_onehot.groupby(['City', 'Borough', 'Neighborhood']).mean().reset_index()
print('The shape of venues_grouped is ', venues_grouped.shape)
venues_grouped.head()

The shape of venues_grouped is  (406, 461)


,City,Borough,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Me

Now with the data frame ```venues_grouped``` at hand, we have finished data collection and data preprocessing. Now we are ready to proceed to perform our data analysis.

## 3. Methodology

In this section, we carry out the main analysis of this project. The procedure can be divided into three steps. First, we build the data frame of user features, ```UserFeatures``` data frame. Second, we build the data frame of features of neighborhoods in New York and Toronto, ```NewNeighborFeatures``` dataframe. Third, we build the data frame providing recommendation, ```RecommendedNeighborhoods``` data frame.

### 3.1 Building the Data Frame of User Features: ```UserFeatures```

We split the data frame ```venues_grouped``` into two daughter data frames. The first one is for neighborhoods in Sha Tin, Hong Kong, named ```UserVenues```. Below is the first 14 columns of the data frame ```UserVenues```.

In [26]:
User_Data = Sha_Tin_Data
UserVenues = venues_grouped[venues_grouped['Neighborhood'].isin(User_Data['Neighborhood'])].reset_index(drop=True)
print('The UserVenues has shape: ', UserVenues.shape)
UserVenues

The UserVenues has shape:  (6, 461)


,City,Borough,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Me

We then average over the rows to get a ```UserFeatures``` data frame, which defines the characteristics of my current working and living place. It is used essentially as a user profile. Below is the first 20 rows of the list ```UserFeatures```.

In [27]:
UserFeatures = UserVenues.drop(['City','Borough','Neighborhood'], 1).mean()
UserFeatures

Accessories Store                           0.000000
Adult Boutique                              0.000000
Afghan Restaurant                           0.000000
African Restaurant                          0.000000
Airport                                     0.000000
Airport Food Court                          0.000000
Airport Gate                                0.000000
Airport Lounge                              0.000000
Airport Service                             0.000000
Airport Terminal                            0.000000
American Restaurant                         0.000000
Animal Shelter                              0.000000
Antique Shop                                0.000000
Aquarium                                    0.000000
Arcade                                      0.000000
Arepa Restaurant                            0.000000
Argentinian Restaurant                      0.000000
Art Gallery                                 0.000000
Art Museum                                  0.

### 3.2 Building the Data Frame of Features of Neighborhoods in New York and Toronto: ```NewNeighborVenues```

The second daughter data frame of ```venues_grouped``` is for neighborhoods in New York and Toronto, named ```NewNeighborVenues```. Below is the first five rows and first 14 columns of the data frame ```NewNeighborVenues```.

In [28]:
NewNeighborVenues = venues_grouped[venues_grouped['Neighborhood'].isin(TwoCities['Neighborhood'])].reset_index(drop=True)
print('The NewNeighborVenues has shape: ', NewNeighborVenues.shape)
NewNeighborVenues.head()

The NewNeighborVenues has shape:  (400, 461)


,City,Borough,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Me

From it, we drop the columns of city, borough, and neighborhood to obtain the data frame ```NewNeighborFeatures```. The first five rows and first 14 columns of the data frame ```NewNeighborFeatures``` are as follows.

In [29]:
NewNeighborFeatures = NewNeighborVenues.drop(['City','Borough','Neighborhood'], 1)
NewNeighborFeatures.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Me

### 3.3 Providing Recommendation: ```RecommendedNeighborhoods```

With data frames ```UserFeatures``` and ```NewNeighborFeatures``` at hand, we are ready to calculate the overall similarity score of each neighborhood in New York and Toronto. We define the similarity score of a neighborhood as the average of features of that neighborhood weighted by the user features. The implementation of this calculation is straight forward. After implementing the calculation and sorting the results, we obtain the resulting data frame ```NewNeighborRating```. The first 20 rows of ```NewNeighborRating``` are the recommended neighborhoods that are most similar to my current working and living place.

In [30]:
#Multiply NewNeighborFeatures by UserFeatures and then take the weighted average
recommendation = (NewNeighborFeatures * UserFeatures).sum(axis=1)/(UserFeatures.sum())
print('The shape of recommendation is: ', recommendation.shape)
recommendation.head()

The shape of recommendation is:  (400,)


0    0.010537
1    0.010909
2    0.012438
3    0.007153
4    0.015333
dtype: float64

In [31]:
# Append the names of cities, boroughs and neighborhoods to recommendation
NewNeighborRating = pd.concat([NewNeighborVenues['City'], NewNeighborVenues['Borough'], NewNeighborVenues['Neighborhood'], recommendation], axis=1, sort=False)
NewNeighborRating.rename(columns={NewNeighborRating.columns[3]:'Similarity Score'}, inplace=True) 
NewNeighborRating.head()

,City,Borough,Neighborhood,Similarity Score
0,New York,Bronx,Allerton,0.010537
1,New York,Bronx,Baychester,0.010909
2,New York,Bronx,Bedford Park,0.012438
3,New York,Bronx,Belmont,0.007153
4,New York,Bronx,Bronxdale,0.015333


In [32]:
NewNeighborRating.sort_values(by=['Similarity Score'], ascending=False, inplace=True)
NewNeighborRating.reset_index(drop=True, inplace=True)
print('The shape of NewNeighborRating is:', NewNeighborRating.shape)
NewNeighborRating.head()

The shape of NewNeighborRating is: (400, 4)


,City,Borough,Neighborhood,Similarity Score
0,Toronto,Scarborough,"Malvern, Rouge",0.082222
1,Toronto,North York,Bayview Village,0.055000
2,Toronto,Scarborough,Woburn,0.037500
3,Toronto,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",0.035171
4,Toronto,North York,Parkwoods,0.032037


## 4. Results

We have these rows of neighborhoods joined with their latitude data and longitude data, forming the data frame of our final results, ```RecommendedNeighborhoods```. This final results data frame is shown below.

In [33]:
RecommendedNeighborhoods = NewNeighborRating.head(20)
RecommendedNeighborhoods

,City,Borough,Neighborhood,Similarity Score
0,Toronto,Scarborough,"Malvern, Rouge",0.082222
1,Toronto,North York,Bayview Village,0.055000
2,Toronto,Scarborough,Woburn,0.037500
3,Toronto,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",0.035171
4,Toronto,North York,Parkwoods,0.032037
5,Toronto,Scarborough,"Birch Cliff, Cliffside West",0.031111
6,Toronto,Downtown Toronto,Christie,0.030131
7,Toronto,Scarborough,"Steeles West, L'Amoreaux West",0.030040
8,Toronto,East York,East Toronto,0.029630
9,Toronto,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.028889


In [34]:
# Add two additional columns to RecommendedNeighborhoods
Place = TwoCities[TwoCities['Neighborhood'].isin(RecommendedNeighborhoods['Neighborhood'])]
RecommendedNeighborhoods = RecommendedNeighborhoods.merge(Place, on=['City', 'Borough', 'Neighborhood'], how='left')
RecommendedNeighborhoods

,City,Borough,Neighborhood,Similarity Score,Latitude,Longitude
0,Toronto,Scarborough,"Malvern, Rouge",0.082222,43.806686,-79.194353
1,Toronto,North York,Bayview Village,0.055000,43.786947,-79.385975
2,Toronto,Scarborough,Woburn,0.037500,43.770992,-79.216917
3,Toronto,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",0.035171,43.605647,-79.501321
4,Toronto,North York,Parkwoods,0.032037,43.753259,-79.329656
5,Toronto,Scarborough,"Birch Cliff, Cliffside West",0.031111,43.692657,-79.264848
6,Toronto,Downtown Toronto,Christie,0.030131,43.669542,-79.422564
7,Toronto,Scarborough,"Steeles West, L'Amoreaux West",0.030040,43.799525,-79.318389
8,Toronto,East York,East Toronto,0.029630,43.685347,-79.338106
9,Toronto,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.028889,43.643515,-79.577201


**Our final results, the 20 recommended neighborhoods are shown in the above data frame.** Here we visualize these neighborhoods using the Folium map. The recommended neighborhoods in New York and Toronto are shown in red dots below.

In [35]:
map_newyork = folium.Map(location=[latitude_NY, longitude_NY], zoom_start=10)

for lat, lng, borough, neighborhood in zip(RecommendedNeighborhoods['Latitude'], RecommendedNeighborhoods['Longitude'], RecommendedNeighborhoods['Borough'], RecommendedNeighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [36]:
map_TRT = folium.Map(location=[latitude_TRT, longitude_TRT], zoom_start=10)

for lat, lng, borough, neighborhood in zip(RecommendedNeighborhoods['Latitude'], RecommendedNeighborhoods['Longitude'], RecommendedNeighborhoods['Borough'], RecommendedNeighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TRT)  
    
map_TRT

## 4. Discussion

We count the number of recommended neighborhoods in New York and Toronto respectively. It turns out that there are 4 recommended neighborhoods in New York and 16 in Toronto. That is a ratio of 1:4. We recall that the total number of neighborhoods in New York is roughly three times the number of neighborhoods in Toronto. We should get a ratio of about 3:1 if the recommended neighborhoods were randomly chosen.

In [37]:
RecommendedNeighborhoods.groupby('City').count()

,Borough,Neighborhood,Similarity Score,Latitude,Longitude
City,,,,,
New York,4,4,4,4,4
Toronto,16,16,16,16,16


Furthermore, we notice that all of the first ten recommended neighborhoods, with top ten highest similarity scores, are neighborhoods in Toronto. **Thus, we conclude that Toronto is much more similar to Sha Tin, Hong Kong than New York.**

The neighborhood Malvern-Rouge in Scarborough has a similarity score significantly higher than any of the rest of neighborhoods. **Thus, we recommend Malvern-Rouge, Scarborough, Toronto as the most familiar neighborhood.**

Next, we want to gain some intuition about the recommended neighborhoods. To do so, we find out the top ten most common venue categories in each recommended neighborhood and use the data to form the data frame ```Discussion```.

In [38]:
RecommendedNeighborhoods

,City,Borough,Neighborhood,Similarity Score,Latitude,Longitude
0,Toronto,Scarborough,"Malvern, Rouge",0.082222,43.806686,-79.194353
1,Toronto,North York,Bayview Village,0.055000,43.786947,-79.385975
2,Toronto,Scarborough,Woburn,0.037500,43.770992,-79.216917
3,Toronto,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",0.035171,43.605647,-79.501321
4,Toronto,North York,Parkwoods,0.032037,43.753259,-79.329656
5,Toronto,Scarborough,"Birch Cliff, Cliffside West",0.031111,43.692657,-79.264848
6,Toronto,Downtown Toronto,Christie,0.030131,43.669542,-79.422564
7,Toronto,Scarborough,"Steeles West, L'Amoreaux West",0.030040,43.799525,-79.318389
8,Toronto,East York,East Toronto,0.029630,43.685347,-79.338106
9,Toronto,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.028889,43.643515,-79.577201


In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10
TheDataFrame = NewNeighborVenues

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = TheDataFrame['Neighborhood']

for ind in np.arange(TheDataFrame.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TheDataFrame.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Supermarket,Pharmacy,Martial Arts Dojo,Gas Station,Check Cashing Service,Donut Shop,Chinese Restaurant,Fried Chicken Joint
1,Baychester,Donut Shop,Moving Target,Fried Chicken Joint,Discount Store,Pet Store,Men's Store,Mexican Restaurant,Sandwich Place,Arcade,Bus Station
2,Bedford Park,Diner,Chinese Restaurant,Deli / Bodega,Mexican Restaurant,Fried Chicken Joint,Pizza Place,Sandwich Place,Pharmacy,Supermarket,Park
3,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Grocery Store,Donut Shop,Bank,Liquor Store,Shoe Store
4,Bronxdale,Paper / Office Supplies Store,Supermarket,Mexican Restaurant,Performing Arts Venue,Chinese Restaurant,Deli / Bodega,Italian Restaurant,Gym,Pizza Place,Park


In [41]:
# Add additional columns to RecommendedNeighborhoods
Place_Discussion = neighborhoods_venues_sorted[neighborhoods_venues_sorted['Neighborhood'].isin(RecommendedNeighborhoods['Neighborhood'])]
Discussion = RecommendedNeighborhoods.merge(Place_Discussion, on=['Neighborhood'], how='left')
Discussion

,City,Borough,Neighborhood,Similarity Score,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Toronto,Scarborough,"Malvern, Rouge",0.082222,43.806686,-79.194353,Fast Food Restaurant,Yoga Studio,Exhibit,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant
1,Toronto,North York,Bayview Village,0.055000,43.786947,-79.385975,Chinese Restaurant,Bank,Japanese Restaurant,Café,Yoga Studio,Event Space,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
2,Toronto,Scarborough,Woburn,0.037500,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Soccer Field,Yoga Studio,Dosa Place,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant
3,Toronto,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",0.035171,43.605647,-79.501321,Café,Gym,Fried Chicken Joint,Liquor Store,Seafood Restaurant,Pharmacy,Coffee Shop,Restaurant,Pizza Place,American Restaurant
4,Toronto,North York,Parkwoods,0.032037,43.753259,-79.329656,Food & Drink Shop,Park,Fast Food Restaurant,Event Space,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant
5,Toronto,Scarborough,"Birch Cliff, Cliffside West",0.031111,43.692657,-79.264848,Skating Rink,College Stadium,Café,General Entertainment,Yoga Studio,Event Service,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant
6,Toronto,Downtown Toronto,Christie,0.030131,43.669542,-79.422564,Grocery Store,Café,Park,Diner,Coffee Shop,Restaurant,Athletics & Sports,Candy Store,Italian Restaurant,Nightclub
7,Toronto,Scarborough,"Steeles West, L'Amoreaux West",0.030040,43.799525,-79.318389,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Grocery Store,Breakfast Spot,Nail Salon,Burger Joint,Pharmacy,Bank,Pizza Place
8,Toronto,East York,East Toronto,0.029630,43.685347,-79.338106,Convenience Store,Park,Coffee Shop,Event Space,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant
9,Toronto,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.028889,43.643515,-79.577201,Beer Store,Pizza Place,Shopping Plaza,Café,Coffee Shop,Liquor Store,Cosmetics Shop,Filipino Restaurant,Ethiopian Restaurant,Dumpling Restaurant


The information of the recommended neighborhoods revealed by the above data frame is consistent with our intuition. As we can see, in most of the top ten most recommended neighborhoods, there are many Asian-style restaurants, including dumpling restaurants, Chinese restaurants, Japanese restaurants, and Korean restaurants. This is reasonable and in line with expectations because I used data of Sha Tin, Hong Kong, which is a borough in East Asia, to create the user profile. There are indeed many Asian-style restaurants in Sha Tin, Hong Kong.

In addition, we may recall that Sha Tin is not close to the city center of Hong Kong. We can observe that among the top ten most recommended neighborhoods in Toronto, nine of these neighborhoods are not close to the city center of Toronto. The exception is the neighborhood of Christie located at Downtown Toronto. Among the four recommended neighborhoods in New York, none is close to the city center of New York. All of these observations are consistent with intuition, thus suggesting that the algorithm we have built works well.

## 6. Conclusion

In this project, I have built a recommendation system that recommends neighborhoods in given new cities which are most similar to the benchmark location. I have chosen the benchmark location to be Sha Tin district of Hong Kong and have chosen two new cities which are New York and Toronto. The recommendation system returns 20 neighborhoods in these two cities based on their similarity scores with respect to the benchmark location. 

Among these recommended 20 neighborhoods, 16 are in Toronto and four are in New York. If we restrict our attention to the top ten most recommended neighborhoods, we find that they are all in Toronto. Since the number of neighborhoods in New York is roughly three times the number of neighborhoods in Toronto, we conclude that when venue categories are concerned, Toronto is much more similar to Sha Tin, Hong Kong than New York.

The top one most recommended neighborhood is Malvern-Rouge, Scarborough, Toronto. The five most common venue categories in this neighborhood are: fast food restaurant, yoga studio, exhibit, dumpling restaurant, and duty-free shop.

In the future, the analysis done here can be expanded to include more cities. One general commercial application might be a more comprehensive recommendation system which includes all major cities around the world. In order to accomplish this goal, we will need to maintain our own database which stores all the location data for neighborhoods in different cities. We will also need a commercial account on Foursquare so that we can fetch unlimited amount of venue data. 

While doing this project, I have noticed that Foursquare does not always return the same venue information for the same input. Thus, for each time we run the same code, the result may be different slightly. To handle this issue, it would be good that we use our own database to store all the venue data returned by Foursquare every time, thus forming a complete data set of information of venues.

**This concludes my project report. Thank you for reading it!**

**Author: Zicao Fu**